# Rewrite [nofreewill](https://www.kaggle.com/nofreewill)'s [notebook](https://www.kaggle.com/nofreewill/normalize-your-predictions) and make it can run on Kaggle.

!!! The LB score may get worse.

In [ ]:
!conda install -y -c rdkit rdkit

In [ ]:
%%writefile normalize_inchis.py

from tqdm import tqdm
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from pathlib import Path

def normalize_inchi(inchi):
    try:
        mol = Chem.MolFromInchi(inchi)
    except:
        pass
    if mol is None:
        return inchi
    else:
        try: return Chem.MolToInchi(mol)
        except: return inchi
        
submission_name = '../input/bmsmt-ds-model/submission_3.06.csv'
norm_path = Path('submission_norm.csv')

# Do the job
N = norm_path.read_text().count('\n') if norm_path.exists() else 0
print(N, 'number of predictions already normalized')

r = open(submission_name, 'r')
write_mode = 'w' if N == 0 else 'a'
w = open(str(norm_path), write_mode, buffering=1)

for _ in range(N):
    r.readline()
line = r.readline()  # this line is the header or is where it died last time
w.write(line)

pbar = tqdm()
while True:
    line = r.readline()
    if not line:
        break  # done
    image_id = line.split(',')[0]
    inchi = ','.join(line[:-1].split(',')[1:]).replace('"','')
    inchi_norm = normalize_inchi(inchi)
    w.write(f'{image_id},"{inchi_norm}"\n')
    pbar.update(1)

r.close()
w.close()

In [ ]:
!ls

In [ ]:
!while [ 1 ]; do python normalize_inchis.py && break; done

In [ ]:
# import pandas as pd
# import Levenshtein
# from tqdm import tqdm

# sub_df = pd.read_csv(submission_name)
# sub_norm_df = pd.read_csv(norm_path)

# lev = 0
# N = len(sub_df)
# for i in tqdm(range(N)):
#     inchi, inchi_norm = sub_df.iloc[i,1], sub_norm_df.iloc[i,1]
#     lev += Levenshtein.distance(inchi, inchi_norm)

# print(lev/N)